# Atlas Vector Search - Create Embeddings - Open Source - Existing Data

This notebook is a companion to the [Create Embeddings](https://www.mongodb.com/docs/atlas/atlas-vector-search/create-embeddings/) page. Refer to the page for set-up instructions and detailed explanations.

This notebook takes you through how to generate embeddings from **existing data in Atlas** by using the open-source ``nomic-embed-text-v1`` model. It also includes code to convert your embeddings to BSON binData vectors for efficient processing of your data.

<a target="_blank" href="https://colab.research.google.com/github/mongodb/docs-notebooks/blob/main/create-embeddings/open-source-existing-data.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [ ]:
pip install --quiet --upgrade sentence-transformers pymongo einops

In [ ]:
from sentence_transformers import SentenceTransformer

# Load the embedding model
model = SentenceTransformer("nomic-ai/nomic-embed-text-v1", trust_remote_code=True)

# Define a function to generate embeddings in multiple precisions
def get_embedding(data, precision="float32"):
    return model.encode(data, precision=precision)

In [ ]:
from bson.binary import Binary

# Generate BSON vector using `BinaryVectorDtype`
def generate_bson_vector(vector, vector_dtype):
    return Binary.from_vector(vector, vector_dtype)

In [ ]:
# Function to create documents with BSON vector embeddings
def create_docs_with_bson_vector_embeddings(bson_float32, bson_int8, bson_int1, data):
  docs = []
  for i, (bson_f32_emb, bson_int8_emb, bson_int1_emb, text) in enumerate(zip(bson_float32, bson_int8, bson_int1, data)):
        doc = {
            "_id": i,
            "data": text,
            "BSON-Float32-Embedding": bson_f32_emb,
            "BSON-Int8-Embedding": bson_int8_emb,
            "BSON-Int1-Embedding": bson_int1_emb,
        }
        docs.append(doc)
  return docs

In [ ]:
# Example generating embeddings for the strings "foo" and "bar"
data = ["foo", "bar"]
float32_embeddings = get_embedding(data, "float32")
int8_embeddings = get_embedding(data, "int8")
int1_embeddings = get_embedding(data, "ubinary")

print("Float32 Embedding:", float32_embeddings)
print("Int8 Embedding:", int8_embeddings)
print("Int1 Embedding (binary representation):", int1_embeddings)

In [ ]:
from bson.binary import BinaryVectorDtype

bson_float32_embeddings = []
bson_int8_embeddings = []
bson_int1_embeddings = []

for (f32_emb, int8_emb, int1_emb) in zip(float32_embeddings, int8_embeddings, int1_embeddings):
    bson_float32_embeddings.append(generate_bson_vector(f32_emb, BinaryVectorDtype.FLOAT32))
    bson_int8_embeddings.append(generate_bson_vector(int8_emb, BinaryVectorDtype.INT8))
    bson_int1_embeddings.append(generate_bson_vector(int1_emb, BinaryVectorDtype.PACKED_BIT))

# Print the embeddings
print(f"The converted bson_float32_new_embedding is: {bson_float32_embeddings}")
print(f"The converted bson_int8_new_embedding is: {bson_int8_embeddings}")
print(f"The converted bson_int1_new_embedding is: {bson_int1_embeddings}")

In [ ]:
import pymongo

# Connect to your Atlas cluster
mongo_client = pymongo.MongoClient("<connection-string>")
db = mongo_client["sample_airbnb"]
collection = db["listingsAndReviews"]

# Define a filter to exclude documents with null or empty 'summary' fields
summary_filter = { '$and': [ { 'summary': { '$exists': True, '$ne': None } } ] }

# Get a subset of documents in the collection
documents = collection.find(summary_filter, {'_id': 1, 'summary': 1}).limit(50)

In [ ]:
from pymongo import UpdateOne

# Generate the list of bulk write operations
operations = []
for doc in documents:
    summary = doc["summary"]
    # Generate embeddings for this document
    float32_embeddings = get_embedding(summary, precision="float32")
    int8_embeddings = get_embedding(summary, precision="int8")
    int1_embeddings = get_embedding(summary, precision="ubinary")
    
    # Convert embeddings to BSON vectors
    bson_float32_embeddings = generate_bson_vector(float32_embeddings, BinaryVectorDtype.FLOAT32)
    bson_int8_embeddings = generate_bson_vector(int8_embeddings, BinaryVectorDtype.INT8)
    bson_int1_embeddings = generate_bson_vector(int1_embeddings, BinaryVectorDtype.PACKED_BIT)
    
    # Add the update operation to the list
    operations.append(UpdateOne(
        {"_id": doc["_id"]},
        {"$set": {
            "BSON-Float32-Embedding": bson_float32_embeddings,
            "BSON-Int8-Embedding": bson_int8_embeddings,
            "BSON-Int1-Embedding": bson_int1_embeddings
        }}
    ))

# Execute the bulk write operation
if operations:
    result = collection.bulk_write(operations)
    updated_doc_count = result.modified_count

print(f"Updated {updated_doc_count} documents.")

In [ ]:
from pymongo.operations import SearchIndexModel
import time

# Create your index model, then create the search index
search_index_model = SearchIndexModel(
  definition = {
    "fields": [
      {
        "type": "vector",
        "path": "BSON-Float32-Embedding",
        "similarity": "dotProduct",
        "numDimensions": 768
      },
      {
        "type": "vector",
        "path": "BSON-Int8-Embedding",
        "similarity": "dotProduct",
        "numDimensions": 768
      },
      {
        "type": "vector",
        "path": "BSON-Int1-Embedding",
        "similarity": "euclidean",
        "numDimensions": 768
      }
    ]
  },
  name="vector_index",
  type="vectorSearch",
)
result = collection.create_search_index(model=search_index_model)
print("New search index named " + result + " is building.")

# Wait for initial sync to complete
print("Polling to check if the index is ready. This may take up to a minute.")
predicate=None
if predicate is None:
  predicate = lambda index: index.get("queryable") is True

while True:
  indices = list(collection.list_search_indexes(result))
  if len(indices) and predicate(indices[0]):
    break
  time.sleep(5)
print(result + " is ready for querying.")

In [ ]:
# Prepare your query
query_text = "beach house"

# Generate embedding for the search query
query_float32_embeddings = get_embedding(query_text, precision="float32")
query_int8_embeddings = get_embedding(query_text, precision="int8")
query_int1_embeddings = get_embedding(query_text, precision="ubinary")

# Convert each embedding to BSON format
query_bson_float32_embeddings = generate_bson_vector(query_float32_embeddings, BinaryVectorDtype.FLOAT32)
query_bson_int8_embeddings = generate_bson_vector(query_int8_embeddings, BinaryVectorDtype.INT8)
query_bson_int1_embeddings = generate_bson_vector(query_int1_embeddings, BinaryVectorDtype.PACKED_BIT)

# Define vector search pipeline for each precision
pipelines = []
for query_embedding, path in zip(
    [query_bson_float32_embeddings, query_bson_int8_embeddings, query_bson_int1_embeddings],
    ["BSON-Float32-Embedding", "BSON-Int8-Embedding", "BSON-Int1-Embedding"]
):
    pipeline = [
       {
          "$vectorSearch": {
                "index": "vector_index",  # Adjust if necessary
                "queryVector": query_embedding,
                "path": path,
                "exact": True,
                "limit": 5
          }
       },
       {
          "$project": {
             "_id": 0,
             "summary": 1,
             "score": {
                "$meta": "vectorSearchScore"
             }
          }
       }
    ]
    pipelines.append(pipeline)

# Execute the search for each precision
for pipeline in pipelines:
    print(f"\nResults for {pipeline[0]['$vectorSearch']['path']}:")
    results = collection.aggregate(pipeline)
    
    # Print results
    for i in results:
        print(i)